In [5]:
import pandas as pd

# Load the datasets
survey_data = pd.read_csv('adjusted_reponse_survey.csv')
contract_data = pd.read_csv('Notebooks\experiment\curr_data.csv')

from scipy.stats import wilcoxon, shapiro

# Merge the datasets on player names for analysis
merged_data = contract_data.merge(
    survey_data, how='left', left_on='Player ID', right_on='PLAYER NAME'
)

contract_data.head()


<>:5: SyntaxWarning: invalid escape sequence '\e'
<>:5: SyntaxWarning: invalid escape sequence '\e'
C:\Users\telat\AppData\Local\Temp\ipykernel_21964\839069531.py:5: SyntaxWarning: invalid escape sequence '\e'
  contract_data = pd.read_csv('Notebooks\experiment\curr_data.csv')


,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Realized Retailer Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,23.45,234.6,300,0.561167,77.72,5.28,268.64,415,240,0.607045
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,15.35,347.8,320,0.479185,82.80,8.20,356.60,364,400,0.505135
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,15.35,347.8,320,0.479185,82.80,8.20,356.60,364,124,0.505135
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,24.00,153.0,306,0.666667,72.00,3.00,189.00,450,153,0.704225
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,23.45,234.6,300,0.561167,77.72,5.28,268.64,415,240,0.607045


In [8]:
# Hypothesis 1: Does Risk Aversion affect the difference between actual and optimal stock levels?
# Null Hypothesis (H0): There is no significant difference in the stock-keeping behavior of high-risk-averse and low-risk-averse individuals.
# Alternative Hypothesis (H1): There is a significant difference in the stock-keeping behavior based on risk aversion.

# Separate high and low risk aversion groups (median split)
risk_median = merged_data['Risk Averse Coefficient'].median()
high_risk_averse = merged_data[merged_data['Risk Averse Coefficient'] >= risk_median]
low_risk_averse = merged_data[merged_data['Risk Averse Coefficient'] < risk_median]




# Calculate the difference between realized and optimal stock for both groups
high_risk_diff = high_risk_averse['Stock'] - high_risk_averse['Optimal_Stock']
low_risk_diff = low_risk_averse['Stock'] - low_risk_averse['Optimal_Stock']



Series([], dtype: int64)


In [ ]:
# Conduct Wilcoxon test (since normality might not hold)
_, high_risk_pval = shapiro(high_risk_diff.dropna())
_, low_risk_pval = shapiro(low_risk_diff.dropna())

# Use Wilcoxon if normality is rejected in either group
if high_risk_pval < 0.05 or low_risk_pval < 0.05:
    test_stat, p_value = wilcoxon(high_risk_diff.dropna(), low_risk_diff.dropna(), alternative='two-sided')
else:
    test_stat, p_value = None, "Normality holds in both groups; consider parametric test."

test_stat, p_value